In [3]:
import torch
from torch.utils.data import Dataset
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from tqdm import tqdm
import time
import copy
from collections import Counter

class PCamDataset(Dataset):
    def __init__(self, h5_x_path, h5_y_path, transform=None):
        self.x_h5 = h5py.File(h5_x_path, 'r')
        self.y_h5 = h5py.File(h5_y_path, 'r')
        self.transform = transform
        self.length = len(self.y_h5['y'])

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        image = self.x_h5['x'][idx]  # shape: (96, 96, 3), dtype: uint8
        label = self.y_h5['y'][idx].item()


        # numpy → PIL → transform
        image = image.astype(np.uint8)
        image = transforms.ToPILImage()(image)

        if self.transform:
            image = self.transform(image)

        return image, label

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(96, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# データセットとデータローダーの設定 (あなたのコードをそのまま使用)
train_dataset = PCamDataset('camelyonpatch_level_2_split_train_x.h5',
                           'camelyonpatch_level_2_split_train_y.h5',
                           transform=transform_train)

test_dataset = PCamDataset('camelyonpatch_level_2_split_test_x.h5',
                          'camelyonpatch_level_2_split_test_y.h5',
                          transform=transform_test)

val_dataset = PCamDataset('valid_x_uncompressed.h5', 'valid_y_uncompressed.h5', transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)


In [4]:
from collections import Counter

# ラベルをすべて取得
labels = [train_dataset[i][1] for i in range(len(train_dataset))]

# クラスごとの件数をカウント
class_counts = Counter(labels)

# 合計数
total = sum(class_counts.values())

# 出力
for cls in sorted(class_counts):
    count = class_counts[cls]
    ratio = count / total * 100
    print(f"Class {cls}: {count} samples ({ratio:.2f}%)")


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models, transforms
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm
import time
import copy
import os
from torchvision.models import resnet50, ResNet50_Weights

# ========================
# データ変換
# ========================
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ========================
# データローダー
# ========================
train_dataset = PCamDataset('camelyonpatch_level_2_split_train_x.h5', 'camelyonpatch_level_2_split_train_y.h5', transform=transform_train)
val_dataset   = PCamDataset('valid_x_uncompressed.h5', 'valid_y_uncompressed.h5', transform=transform_test)
test_dataset  = PCamDataset('camelyonpatch_level_2_split_test_x.h5', 'camelyonpatch_level_2_split_test_y.h5', transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

# ========================
# モデル構築
# ========================
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 2)
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pretrained ResNet50 & modify final layer
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 2)  # PCamは2クラス
model = model.to(device)

# ========================
# Focal Loss
# ========================
class FocalLoss(nn.Module):
    def __init__(self, alpha=2, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs, targets):
        logp = self.ce(inputs, targets)
        p = torch.exp(-logp)
        loss = self.alpha * (1 - p) ** self.gamma * logp
        return loss.mean()

# ========================
# 環境設定
# ========================
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = FocalLoss(alpha=2, gamma=2)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# ========================
# 学習関数
# ========================
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}\n----------')

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            dataloader = train_loader if phase == 'train' else val_loader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloader):
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val':
                if epoch_acc > best_val_acc:
                    best_val_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    epochs_no_improve = 0
                    torch.save(model.state_dict(), f'model_valacc_{best_val_acc:.4f}.pth')
                else:
                    epochs_no_improve += 1
                    if epochs_no_improve >= 5:
                        print("Early stopping")
                        model.load_state_dict(best_model_wts)
                        return model

    model.load_state_dict(best_model_wts)
    return model

# ========================
# 学習実行
# ========================
model = train_model(model, criterion, optimizer, scheduler, num_epochs=30)

# ========================
# テスト評価 + クラス別分析
# ========================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)
        preds = (probs[:, 1] > 0.4).long()  # 閾値調整でRecall改善
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 精度・詳細評価
print("\n=== Test Performance ===")
print(f"Test Accuracy: {accuracy_score(all_labels, all_preds):.4f}")
print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds, digits=4))


Epoch 0/29
----------


100%|██████████| 4096/4096 [04:28<00:00, 15.24it/s]


train Loss: 0.1463 Acc: 0.8897


100%|██████████| 512/512 [00:11<00:00, 43.01it/s]


val Loss: 0.1510 Acc: 0.8902
Epoch 1/29
----------


...

train Loss: 0.0854 Acc: 0.9416


100%|██████████| 512/512 [00:11<00:00, 44.46it/s]


val Loss: 0.1433 Acc: 0.9011
Early stopping


100%|██████████| 512/512 [00:11<00:00, 43.86it/s]



=== Test Performance ===
Test Accuracy: 0.8883
[[15048  1343]
 [ 2316 14061]]
              precision    recall  f1-score   support

           0     0.8666    0.9181    0.8916     16391
           1     0.9128    0.8586    0.8849     16377

    accuracy                         0.8883     32768
   macro avg     0.8897    0.8883    0.8882     32768
weighted avg     0.8897    0.8883    0.8882     32768



In [4]:
tta_transforms = [
    transforms.Compose([  # オリジナル
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    transforms.Compose([  # 左右反転
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    transforms.Compose([  # 上下反転
        transforms.RandomVerticalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    transforms.Compose([  # 回転90°
        transforms.RandomRotation(degrees=(90, 90)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
]

from torchvision.transforms import ToPILImage
to_pil = ToPILImage()

# 正規化済みの逆変換（ToPILImage用）
def denormalize_for_pil(tensor):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std  = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    return (tensor * std + mean).clamp(0, 1)

model.eval()
all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        B = images.size(0)

        tta_preds = []

        for t in tta_transforms:
            aug_imgs = []
            for i in range(B):
                img = denormalize_for_pil(images[i].cpu())
                img_pil = to_pil(img)
                img_aug = t(img_pil)  # TTA変換適用
                aug_imgs.append(img_aug)
            batch = torch.stack(aug_imgs).to(device)
            outputs = model(batch)
            probs = torch.softmax(outputs, dim=1)
            tta_preds.append(probs)

        mean_probs = torch.stack(tta_preds).mean(dim=0)
        preds = (mean_probs[:, 1] > 0.4).long()
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(mean_probs[:, 1].cpu().numpy())


from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("\n=== TTA Test Performance ===")
print(f"Test Accuracy: {accuracy_score(all_labels, all_preds):.4f}")
print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds, digits=4))


  0%|          | 0/512 [00:00<?, ?it/s]

100%|██████████| 512/512 [02:19<00:00,  3.68it/s]


=== TTA Test Performance ===
Test Accuracy: 0.8958
[[15128  1263]
 [ 2152 14225]]
              precision    recall  f1-score   support

           0     0.8755    0.9229    0.8986     16391
           1     0.9185    0.8686    0.8928     16377

    accuracy                         0.8958     32768
   macro avg     0.8970    0.8958    0.8957     32768
weighted avg     0.8969    0.8958    0.8957     32768



In [1]:
def fgsm_attack(model, images, labels, epsilon):
    images = images.clone().detach().to(device)
    labels = labels.clone().detach().to(device)
    images.requires_grad = True

    outputs = model(images)
    loss = criterion(outputs, labels)
    model.zero_grad()
    loss.backward()

    # FGSM perturbation: sign(∇_x loss)
    perturbed_images = images + epsilon * images.grad.sign()
    perturbed_images = torch.clamp(perturbed_images, 0, 1)
    return perturbed_images


In [2]:
epsilon = 0.03

model.eval()
adv_imgs = []

for img, label in zip(imgs, labels):
    img = img.unsqueeze(0).to(device)      # (C, H, W) → (1, C, H, W)
    label = label.unsqueeze(0).to(device)  # ラベルもバッチ化

    adv_img = fgsm_attack(model, img, label, epsilon)
    adv_imgs.append(adv_img.detach().cpu())  # detachとCPUに移して蓄積

torch.cuda.empty_cache()  # メモリの断片化も軽減
adv_imgs = torch.cat(adv_imgs)  # (N, C, H, W)

# ここから敵対的画像を用いた推論・正解率計算
correct = 0
total = 0

with torch.no_grad():
    for adv_img, label in zip(adv_imgs, labels):
        adv_img = adv_img.unsqueeze(0).to(device)
        label = label.unsqueeze(0).to(device)

        output = model(adv_img)
        pred = output.argmax(dim=1)
        correct += (pred == label).sum().item()
        total += label.size(0)

acc = 100 * correct / total
print(f'Adversarial Test Accuracy (FGSM, ε={epsilon}): {acc:.2f}%')


NameError: name 'model' is not defined